In [24]:
from sec_api import QueryApi, ExtractorApi
import json
import requests
import pandas as pd
from IPython.display import display, HTML

queryAPI = QueryApi(api_key='041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550')
extractorApi = ExtractorApi(api_key='041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550')

In [25]:
#formType:(“10-K”, “10-KT”, “10KSB”, “10KT405”, “10KSB40”, “10-K405”)
class SEC_QUERY:

    def __init__(self, form_type, ticker, size):
        self.form_type = form_type
        self.ticker = ticker
        self.size = size
        self.query = {
            "query": { 
                "query_string" : {
                    "query": f"formType:\"{self.form_type}\" AND ticker:{self.ticker}",
                }
            },
            "from": "0",
            "size": size,
            "sort": [{"filedAt": {"order": "desc"}}]
        }
    
    def get_response(self, index:int):
        response = queryAPI.get_filings(self.query)
        return json.dumps(response["filings"][index], indent=2)
    

    def extract(self, index:int):
        query_str = self.get_response(index)
        q_dict = json.loads(query_str)
        filing_url = q_dict['linkToHtml']

        xbrl_converter_api_endpoint = "https://api.sec-api.io/xbrl-to-json"
        api_key = '041c38e656c459b1b336a7bea7d83c057482f669d2e663a1b6fa8864b2071550'

        final_url = xbrl_converter_api_endpoint + "?htm-url=" + filing_url + "&token=" + api_key
        response = requests.get(final_url)

        final_json = json.loads(response.text)
        return final_json, filing_url

In [27]:
query1 = SEC_QUERY("10-K", "TSLA", "10")
#print(query1.get_response(0))
init_query, f_url = query1.extract(0)

# 10-K supported items [1, 1A, 1B, 2, 3, 4, 5, 6, 7, 7A, 8, 9, 9A, 9B, 10, 11, 12, 13, 14, 15]
section_text = extractorApi.get_section(f_url, "8", "text").split(' ')
section_html = extractorApi.get_section(f_url, "8", 'html')

In [28]:
display(HTML(section_html[0:150000]))

['', 'ITEM', '8.', 'FINANCIAL', 'STATEMENTS', 'AND', 'SUPPLEMENTARY', 'DATA', '\n\nIndex', 'to', 'Consolidated', 'Financial', 'Statements', '\n\n##TABLE_START', 'Page', '\n\nReport', 'of', 'Independent', 'Registered', 'Public', 'Accounting', 'Firm', '(PCAOB', 'ID:', '238', ')', '\n\nConsolidated', 'Balance', 'Sheets', '\n\nConsolidated', 'Statements', 'of', 'Operations', '\n\nConsolidated', 'Statements', 'of', 'Comprehensive', 'Income', '\n\nConsolidated', 'Statements', 'of', 'Redeemable', 'Noncontrolling', 'Interests', 'and', 'Equity', '\n\nConsolidated', 'Statements', 'of', 'Cash', 'Flows', '\n\nNotes', 'to', 'Consolidated', 'Financial', 'Statements', '\n\n##TABLE_END\n\nReport', 'of', 'Independent', 'Registered', 'Public', 'Accounting', 'Firm', '\n\nTo', 'the', 'Board', 'of', 'Directors', 'and', 'Stockholders', 'of', 'Tesla,', 'Inc.', '\n\nOpinions', 'on', 'the', 'Financial', 'Statements', 'and', 'Internal', 'Control', 'over', 'Financial', 'Reporting', '\n\nWe', 'have', 'audited', '

In [29]:
tables = pd.read_html(section_html)

In [30]:
tables

[                                                   0  \
 0                                                NaN   
 1                                                NaN   
 2  Report of Independent Registered Public Accoun...   
 3                        Consolidated Balance Sheets   
 4              Consolidated Statements of Operations   
 5    Consolidated Statements of Comprehensive Income   
 6  Consolidated Statements of Redeemable Noncontr...   
 7              Consolidated Statements of Cash Flows   
 8         Notes to Consolidated Financial Statements   
 
                                                    1  \
 0                                                NaN   
 1                                                NaN   
 2  Report of Independent Registered Public Accoun...   
 3                        Consolidated Balance Sheets   
 4              Consolidated Statements of Operations   
 5    Consolidated Statements of Comprehensive Income   
 6  Consolidated Statements o